# AruCo marker video for multi-cam synchnonization

This is a simplified version of [GoPro Precision Date and Time (Local)](https://gopro.github.io/labs/control/precisiontime/).  Instead of displaying a QR code representing the current time in microseconds, the video in this example shows an AruCo marker at each frame.


In [1]:
import sys, os, cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

class AruCoMaker:
    def __init__(self, aruco_dict, dict_size, square_length):
        self.aruco_dict = aruco_dict
        self.dict_size = dict_size
        self.square_length = square_length
    
    def __getitem__(self, idx):
        idx %= self.dict_size
        return cv2.aruco.generateImageMarker(self.aruco_dict, idx, self.square_length)
    
class ChAruCoDiamondMaker:
    def __init__(self, aruco_dict, dict_size, square_length, marker_length):
        self.aruco_dict = aruco_dict
        self.dict_size = dict_size
        self.square_length = square_length
        self.marker_length = marker_length
    
    def __getitem__(self, idx):
        diamond_marker_ids = np.array([0, 1, 2, 3], dtype=int)
        diamond_marker_ids += idx
        diamond_marker_ids %= self.dict_size
        board = cv2.aruco.CharucoBoard((3, 3), self.square_length, self.marker_length, self.aruco_dict, diamond_marker_ids)
        return board.generateImage((self.square_length, self.square_length))

def padto(image, width, height):
    ih, iw = image.shape[:2]
    buf = np.ones((height, width, 3), dtype=np.uint8) * 255
    dx = (width-iw)//2
    dy = (height-ih)//2
    buf[dy:dy+ih,dx:dx+iw,:] = image[:,:,None]
    return buf


In [11]:
from IPython.display import Video

duration = 30   # 30 sec
video_width = 1920
video_height = 1080

aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
aruco_dict_size = 250

video_fps = [15, 30]
output_base = ["output-aruco", "output-diamond"]
maker = [AruCoMaker(aruco_dict, aruco_dict_size, 800),
         ChAruCoDiamondMaker(aruco_dict, aruco_dict_size, 750, 450)]

# opencv-python by pip does not suppert 'h264'.
codec = cv2.VideoWriter_fourcc(*'mp4v')


for fps in video_fps:
    for ba, ma in zip(output_base, maker):
        video = cv2.VideoWriter(ba + '.mp4', codec, fps, (video_width, video_height))
        for i in tqdm(range(duration*fps), desc=f'{ba}@{fps}Hz'):
            i = i % aruco_dict_size
            buf = padto(ma[i], video_width, video_height)
            cv2.putText(buf, f'DICT_6X6_250 #{i} @ {fps}Hz', (100, video_height-50), cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 0), 5, cv2.LINE_AA)
            video.write(buf)
        video.release()

        # We can then use ffmpeg to encode MP4V to H264. This makes the filesize smaller in general, and also allows embedding the video in the web browser.
        #!ffmpeg -loglevel quiet -i "{ba}.mp4" -c:v libx265 -crf 22 -tag:v hvc1 -y "{ba}-{fps}hz.mp4"
        #Video('{ba}-{fps}hz.mp4', embed=True)


output-aruco@15Hz:   0%|          | 0/450 [00:00<?, ?it/s]

x265 [info]: HEVC encoder version 3.5+1-f0c1022b6
x265 [info]: build info [Linux][GCC 8.3.0][64 bit] 8bit+10bit+12bit
x265 [info]: using cpu capabilities: MMX2 SSE2Fast LZCNT SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
x265 [info]: Main profile, Level-4 (Main tier)
x265 [info]: Thread pool created using 20 threads
x265 [info]: Slices                              : 1
x265 [info]: frame threads / pool features       : 4 / wpp(17 rows)
x265 [info]: Coding QT: max CU size, min CU size : 64 / 8
x265 [info]: Residual QT: max TU size, max depth : 32 / 1 inter / 1 intra
x265 [info]: ME / range / subpel / merge         : hex / 57 / 2 / 3
x265 [info]: Keyframe min / max / scenecut / bias  : 15 / 250 / 40 / 5.00 
x265 [info]: Lookahead / bframes / badapt        : 20 / 4 / 2
x265 [info]: b-pyramid / weightp / weightb       : 1 / 1 / 0
x265 [info]: References / ref-limit  cu / depth  : 3 / off / on
x265 [info]: AQ: mode / str / qg-size / cu-tree  : 2 / 1.0 / 32 / 1
x265 [info]: Rate Control / qCompress        

output-diamond@15Hz:   0%|          | 0/450 [00:00<?, ?it/s]

x265 [info]: HEVC encoder version 3.5+1-f0c1022b6
x265 [info]: build info [Linux][GCC 8.3.0][64 bit] 8bit+10bit+12bit
x265 [info]: using cpu capabilities: MMX2 SSE2Fast LZCNT SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
x265 [info]: Main profile, Level-4 (Main tier)
x265 [info]: Thread pool created using 20 threads
x265 [info]: Slices                              : 1
x265 [info]: frame threads / pool features       : 4 / wpp(17 rows)
x265 [info]: Coding QT: max CU size, min CU size : 64 / 8
x265 [info]: Residual QT: max TU size, max depth : 32 / 1 inter / 1 intra
x265 [info]: ME / range / subpel / merge         : hex / 57 / 2 / 3
x265 [info]: Keyframe min / max / scenecut / bias  : 15 / 250 / 40 / 5.00 
x265 [info]: Lookahead / bframes / badapt        : 20 / 4 / 2
x265 [info]: b-pyramid / weightp / weightb       : 1 / 1 / 0
x265 [info]: References / ref-limit  cu / depth  : 3 / off / on
x265 [info]: AQ: mode / str / qg-size / cu-tree  : 2 / 1.0 / 32 / 1
x265 [info]: Rate Control / qCompress        

output-aruco@30Hz:   0%|          | 0/900 [00:00<?, ?it/s]

x265 [info]: HEVC encoder version 3.5+1-f0c1022b6
x265 [info]: build info [Linux][GCC 8.3.0][64 bit] 8bit+10bit+12bit
x265 [info]: using cpu capabilities: MMX2 SSE2Fast LZCNT SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
x265 [info]: Main profile, Level-4 (Main tier)
x265 [info]: Thread pool created using 20 threads
x265 [info]: Slices                              : 1
x265 [info]: frame threads / pool features       : 4 / wpp(17 rows)
x265 [info]: Coding QT: max CU size, min CU size : 64 / 8
x265 [info]: Residual QT: max TU size, max depth : 32 / 1 inter / 1 intra
x265 [info]: ME / range / subpel / merge         : hex / 57 / 2 / 3
x265 [info]: Keyframe min / max / scenecut / bias  : 25 / 250 / 40 / 5.00 
x265 [info]: Lookahead / bframes / badapt        : 20 / 4 / 2
x265 [info]: b-pyramid / weightp / weightb       : 1 / 1 / 0
x265 [info]: References / ref-limit  cu / depth  : 3 / off / on
x265 [info]: AQ: mode / str / qg-size / cu-tree  : 2 / 1.0 / 32 / 1
x265 [info]: Rate Control / qCompress        

output-diamond@30Hz:   0%|          | 0/900 [00:00<?, ?it/s]

x265 [info]: HEVC encoder version 3.5+1-f0c1022b6
x265 [info]: build info [Linux][GCC 8.3.0][64 bit] 8bit+10bit+12bit
x265 [info]: using cpu capabilities: MMX2 SSE2Fast LZCNT SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
x265 [info]: Main profile, Level-4 (Main tier)
x265 [info]: Thread pool created using 20 threads
x265 [info]: Slices                              : 1
x265 [info]: frame threads / pool features       : 4 / wpp(17 rows)
x265 [info]: Coding QT: max CU size, min CU size : 64 / 8
x265 [info]: Residual QT: max TU size, max depth : 32 / 1 inter / 1 intra
x265 [info]: ME / range / subpel / merge         : hex / 57 / 2 / 3
x265 [info]: Keyframe min / max / scenecut / bias  : 25 / 250 / 40 / 5.00 
x265 [info]: Lookahead / bframes / badapt        : 20 / 4 / 2
x265 [info]: b-pyramid / weightp / weightb       : 1 / 1 / 0
x265 [info]: References / ref-limit  cu / depth  : 3 / off / on
x265 [info]: AQ: mode / str / qg-size / cu-tree  : 2 / 1.0 / 32 / 1
x265 [info]: Rate Control / qCompress        